# 1. Acquire and Understand Data

## Goal
The goal of this notebook is to obtain the College Football box score data, understand its structure and provenance, and prepare a "tidy" dataset for further analysis.

## 1. Data Acquisition
The data source is a collection of College Football box scores from 2002 to 2024. This dataset includes game-level statistics for both home and away teams, including scoring by quarter, offensive stats (passing, rushing), and turnovers.

**Source File:** `cfb_box-scores_2002-2024 (1).csv`
**Origin:** Likely aggregated from the CollegeFootballData API or a similar sports statistics repository.

In [1]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv("cfb_box-scores_2002-2024 (1).csv")

# Display the first few rows
df.head()

,season,week,date,time_et,game_type,away,home,rank_away,rank_home,conf_away,...,int_away,int_home,pen_num_away,pen_yards_away,pen_num_home,pen_yards_home,possession_away,possession_home,attendance,tv
0,2002,1.0,2002-08-22,7:30 PM,regular,Colorado State,Virginia,NaN,NaN,mwc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57120.0,NaN
1,2002,1.0,2002-08-23,8:00 PM,regular,Fresno State,Wisconsin,NaN,25.0,wac,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75136.0,NaN
2,2002,1.0,2002-08-24,2:30 PM,regular,Texas Tech,Ohio State,NaN,13.0,big12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100037.0,NaN
3,2002,1.0,2002-08-24,4:30 PM,regular,New Mexico,NC State,NaN,NaN,mwc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47018.0,NaN
4,2002,1.0,2002-08-24,7:45 PM,regular,Arizona State,Nebraska,NaN,10.0,pac12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77779.0,NaN


## 2. Data Dictionary (COLS Table)
Below is a description of the key features in the dataset.

In [2]:
cols_info = pd.DataFrame({
    'Column Name': df.columns,
    'Data Type': df.dtypes.values
})
cols_info.head(20)

,Column Name,Data Type
0,season,int64
1,week,float64
2,date,object
3,time_et,object
4,game_type,object
5,away,object
6,home,object
7,rank_away,float64
8,rank_home,float64
9,conf_away,object


## 3. Data Cleaning and Tidy Form
The raw data has one row per game, with columns for both 'home' and 'away' teams. For our analysis (comparing play styles of teams), it is more convenient to have **one row per team-game**.

We will transform the data into a "tidy" format where each observation is a single team's performance in a specific game.

In [3]:
# Split into Home and Away dataframes
home_cols = ['season', 'week', 'date', 'home', 'conf_home', 'score_home', 'score_away', 'pass_att_home', 'pass_yards_home', 'rush_att_home', 'rush_yards_home', 'attendance']
away_cols = ['season', 'week', 'date', 'away', 'conf_away', 'score_away', 'score_home', 'pass_att_away', 'pass_yards_away', 'rush_att_away', 'rush_yards_away', 'attendance']

home = df[home_cols].copy()
home.columns = ['season', 'week', 'date', 'team', 'conference', 'points', 'opp_points', 'pass_att', 'pass_yards', 'rush_att', 'rush_yards', 'attendance']
home['is_home'] = True

away = df[away_cols].copy()
away.columns = ['season', 'week', 'date', 'team', 'conference', 'points', 'opp_points', 'pass_att', 'pass_yards', 'rush_att', 'rush_yards', 'attendance']
away['is_home'] = False

# Combine into a single tidy dataframe
team_games = pd.concat([home, away], ignore_index=True)

# Convert date to datetime
team_games['date'] = pd.to_datetime(team_games['date'])

# Calculate Win/Loss
team_games['win'] = (team_games['points'] > team_games['opp_points']).astype(int)

# Calculate Total Plays and Pass Rate
team_games['total_plays'] = team_games['pass_att'] + team_games['rush_att']
team_games['pass_rate'] = team_games['pass_att'] / team_games['total_plays']

# Filter out garbage time / low data games (optional, e.g., 0 plays)
team_games = team_games[team_games['total_plays'] > 0]

print(f"Total Team-Games: {len(team_games)}")
team_games.head()

Total Team-Games: 34656


,season,week,date,team,conference,points,opp_points,pass_att,pass_yards,rush_att,rush_yards,attendance,is_home,win,total_plays,pass_rate
1543,2004,1.0,2004-08-28,Miami (OH),mac,49,0,37.0,292.0,39.0,162.0,NaN,True,1,76.0,0.486842
1544,2004,1.0,2004-08-28,Virginia Tech,acc,13,24,29.0,180.0,34.0,141.0,91665.0,True,0,63.0,0.460317
1546,2004,2.0,2004-09-02,Utah,mwc,41,21,31.0,381.0,45.0,212.0,NaN,True,1,76.0,0.407895
1547,2004,2.0,2004-09-02,Ball State,mac,11,19,33.0,183.0,33.0,64.0,NaN,True,0,66.0,0.500000
1548,2004,2.0,2004-09-02,Western Michigan,mac,42,0,23.0,245.0,38.0,108.0,21643.0,True,1,61.0,0.377049


## 4. Save Tidy Data
We will save this tidy dataset for use in the Exploration notebook.

In [4]:
team_games.to_csv("tidy_team_games.csv", index=False)